# Predict Good commit message or not

In [1]:
! pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests

In [3]:
from google.colab import drive
import pickle
from google.colab import files
import sys
drive.mount('/content/gdrive', force_remount=True)
sys.path.append("/content/gdrive/My Drive/<path_to_file>")

Mounted at /content/gdrive


In [4]:
import transformers
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

train_filename =  "/content/gdrive/My Drive/test/sampled_messages.csv"
df_train = pd.read_csv(train_filename, encoding='latin-1')

In [6]:
df_train.head(5)

,id,repo_id,label,url,if_mulit_commit,message,new_message1,authorEmail,commitDate,Unnamed: 9
0,3079,2,0,https://github.com/apache/dubbo/commit/1b2e6dc...,1,remove some magic value (#4752) <enter> <ente...,remove some magic value ( <pr_link> ) <enter>...,developer128,2019-08-06T07:23:45Z,NaN
1,5904,4,0,https://github.com/square/okhttp/commit/2fd0da...,,Update concurrency.md (#6290),Update concurrency.md ( <pr_link> ),developer129,2020-09-30T16:01:02Z,NaN
2,4224,2,0,https://github.com/apache/dubbo/commit/9e9e778...,,Optimize_hessian_desr_performance (#1705),Optimize_hessian_desr_performance ( <pr_link> ),developer130,2018-04-26T10:51:02Z,NaN
3,5382,3,0,https://github.com/square/retrofit/commit/0754...,,Add @PartMap annotation.,Add <iden> annotation.,developer131,2014-05-01T17:16:02Z,NaN
4,6983,4,3,https://github.com/square/okhttp/commit/c310d2...,,workaround broken system dns behaviour,workaround broken system <file_name> behaviour,developer132,2017-11-17T11:02:26Z,NaN


In [7]:
def func(path):
  path = path.replace('\r', '').replace('\n', '')
  r = requests.get(path + '.diff')
  text_diff = r.text.split('\n') 

  main_info = []
  for string in text_diff:
      if len(string) > 0 and (string[0] == '-' or string[0] == '+'):
        main_info.append(string)

  return " <nl> ".join(main_info).replace("---", "mmm").replace("+++", "ppp")

In [8]:
def get_commit_diff():
    return df_train["url"].apply(func)

In [9]:
df_train["new_message1"] = df_train["new_message1"].apply(lambda x: x.replace('<enter>', '$enter').replace('<tab>', '$tab'). \
                                    replace('<url>', '$url').replace('<version>', '$version') \
                                    .replace('<pr_link>', '$pull request').replace('<issue_link>',
                                                                                    '$issue') \
                                    .replace('<otherCommit_link>', '$other commit').replace("<method_name>",
                                                                                            "$method") \
                                    .replace("<file_name>", "$file").replace("<iden>", "$token").replace("<nl>", ""))

We suppose if commit message have answer for why and what that it's good message and we don't need information about text in commit. But if we want to know changes we can concatenate two texts and we get info about info how matches commit changes and message

In [10]:
#df_train["text"] = df_train["new_message1"] + df_train["commit_diff"]

In [11]:
data = df_train[["label", "new_message1"]]
data.columns = ["label", "text"]

In [12]:
data.head()

,label,text
0,0,remove some magic value ( $pull request ) $en...
1,0,Update concurrency.md ( $pull request )
2,0,Optimize_hessian_desr_performance ( $pull requ...
3,0,Add $token annotation.
4,3,workaround broken system $file behaviour


In [13]:
train, test = train_test_split(data, test_size=0.25)

In [14]:
train_labels = train['label'].tolist()
train_text = train['text'].tolist()

test_labels = test['label'].tolist()
test_text = test['text'].tolist()

In [15]:
from transformers import AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
def text_to_id(tokenizer, text_list):
    ids_list = []
    
    for item in text_list:
        encoded_item = tokenizer.encode(item, add_special_tokens=True)
        ids_list.append(encoded_item)
    
    return ids_list

In [17]:
train_text_ids = text_to_id(tokenizer, train_text)
test_text_ids = text_to_id(tokenizer, test_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (599 > 512). Running this sequence through the model will result in indexing errors


In [18]:
def padding_truncating(input_ids_list, max_length):
    processed_input_ids_list = []
    for item in input_ids_list:
        seq_list = []
        
        if len(item) < max_length:
            seq_list = [0] * (max_length - len(item))
            item = item + seq_list
        
        elif len(item) >= max_length:
            item = item[:max_length]
            
        processed_input_ids_list.append(item)
    
    return processed_input_ids_list

In [19]:
train_padding_list = padding_truncating(train_text_ids, max_length=200)
test_padding_list = padding_truncating(test_text_ids, max_length=200)

In [20]:
def get_attention_masks(pad_input_ids_list):
    attention_masks_list = []
    
    for item in pad_input_ids_list:
        
        mask_list = []
        for subitem in item:
            if subitem > 0:
                mask_list.append(1)
            else:
                mask_list.append(0)
        attention_masks_list.append(mask_list)
    
    return attention_masks_list

In [21]:
train_attention_masks = get_attention_masks(train_padding_list)
test_attention_masks = get_attention_masks(test_padding_list)

In [22]:
from sklearn.model_selection import train_test_split

train_padding_list, validation_padding_list, train_labels, validation_labels, train_attention_masks, validation_attention_masks = train_test_split(train_padding_list, train_labels, train_attention_masks, random_state=2020, test_size=0.1)

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

train_inputs = torch.tensor(train_padding_list)
validation_inputs = torch.tensor(validation_padding_list)
test_inputs = torch.tensor(test_padding_list)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
test_labels = torch.tensor(test_labels)

train_masks = torch.tensor(train_attention_masks)
validation_masks = torch.tensor(validation_attention_masks)
test_masks = torch.tensor(test_attention_masks)

In [24]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [25]:
from transformers import AdamW
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",  
     num_labels = 4,     
     output_attentions = False, 
     output_hidden_states = False,
)

model.to(device)
None

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [26]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5,
                  eps = 1e-8
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [27]:
from transformers import get_linear_schedule_with_warmup

epochs = 5

total_steps = len(train_dataloader) * epochs
print("total_steps = {}".format(total_steps))

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

total_steps = 350


In [28]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [29]:
from tqdm import tqdm

In [30]:
import random

seed_val = 12345

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

loss_values = []

for epoch_i in range(epochs):
    print(f'\n======== Epoch {epoch_i + 1} / {epochs} ========')
    total_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs[0]

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            

    loss_values.append(avg_train_loss)

    print(f"\n  Average training loss: {avg_train_loss}")
    print("\n Running Validation...")

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1

    print(f"  Accuracy: {eval_accuracy/nb_eval_steps}")

print("\nFinish")


======== Epoch 1 / 5 ========


70it [00:35,  1.96it/s]



  Average training loss: 0.9298414438962936

 Running Validation...
  Accuracy: 0.7265625

======== Epoch 2 / 5 ========


70it [00:34,  2.01it/s]



  Average training loss: 0.6257032854216439

 Running Validation...
  Accuracy: 0.7395833333333334

======== Epoch 3 / 5 ========


...


  Average training loss: 0.40418727483068195

 Running Validation...
  Accuracy: 0.78125

======== Epoch 4 / 5 ========


70it [00:38,  1.82it/s]



  Average training loss: 0.22716706576091902

 Running Validation...
  Accuracy: 0.7526041666666666

======== Epoch 5 / 5 ========


70it [00:37,  1.85it/s]



  Average training loss: 0.11788877788931132

 Running Validation...
  Accuracy: 0.78125

Finish


In [32]:
model.eval()

predictions , true_labels = [], []
 
idx = 0
correct = 0
for batch in tqdm(test_dataloader):
    idx += 1
    
    batch = tuple(t.to(device) for t in batch)
  
    b_input_ids, b_input_mask, b_labels = batch
  
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred = np.argmax(logits, axis=1)
    correct += (pred == label_ids).sum().item()

print("\nTotal correct = ", correct)
print(f"Test accuracy = {correct / len(test_inputs)}")


100%|██████████| 26/26 [00:05<00:00,  5.04it/s]


Total correct =  320
Test accuracy = 0.774818401937046


## PREDICT NNGEN DATASET


In [33]:
with open("/content/gdrive/My Drive/test/cleaned.test.msg", "r") as f:
    messages = f.readlines()
    messages = [x.replace(' \n', '') for x in messages]

In [34]:
test_data = pd.DataFrame({"messages" : messages})
values_text_ids = text_to_id(tokenizer, test_data["messages"].to_list())
values_padding_list = padding_truncating(values_text_ids, max_length=200)
values_attention_masks = get_attention_masks(values_padding_list)
values_inputs = torch.tensor(values_padding_list).to(device)
values_masks = torch.tensor(values_attention_masks).to(device)

In [35]:
values_data = TensorDataset(values_inputs, values_masks)
values_sampler = SequentialSampler(values_data)
values_dataloader = DataLoader(values_data, sampler=values_sampler, batch_size=batch_size)

In [36]:
batches_prediction = []

for batch in tqdm(values_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch
 
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]  
    logits = logits.detach().cpu().numpy()
    pred = np.argmax(logits, axis=1)
    batches_prediction.append(pred)

100%|██████████| 158/158 [00:32<00:00,  4.92it/s]


In [37]:
prediction = np.array(batches_prediction[0])
for x in batches_prediction[1:]:
  prediction = np.concatenate((prediction, x), axis=0)

In [38]:
test_data["label"] = prediction

  * label = 0 means a commit message contains "Why and What".
  * label = 1 means a commit message  contains "Neither Why nor What".
  * label = 2 means a commit message  contains "No What".
  * label = 3 means a commit message  contains "No Why".

In [39]:
test_data["label_good_commit"] = (test_data["label"]  == 0).apply(int)

In [40]:
test_data

,messages,label,label_good_commit
0,Fix snapshot version,2,0
1,update chagelog,0,1
2,edit coverage colors icon,3,0
3,LRQA - 14419 Add new property to turn on runni...,0,1
4,Added joscar JAR .,3,0
...,...,...,...
2516,reapply fabric puglin,3,0
2517,Bump up revision number .,0,1
2518,Ignore local changes on .,3,0
2519,Don ' t publish shrinkwrap,2,0
